# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [195]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from collections import Counter

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@hugoncalves99***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [200]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)
    data['consumer_key'] = ''
    data['consumer_secret'] = ''
    data['access_token'] = ''
    data['access_token_secret'] = ''

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])


___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [201]:
#Produto escolhido:
produto = 'Ovomaltine'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [202]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [203]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [206]:
# Remove os caracteres de pontuação (',','.',':',etc)
import string
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
        return text

# Lê a base de dados Excel
d = pd.read_excel('Ovomaltine.xlsx')

# Retira caracteres de pontuação
d['Treinamento'] = d['Treinamento'].apply(remove_punctuation)
d['Treinamento']
pd.DataFrame(d['Treinamento'])

# Renomeia as colunas 
d = pd.concat([d['Treinamento'],d['Unnamed: 1']], axis=1)
d.columns = ['Treinamento','Relevância']

# Separa dados relevantes dos irrelevantes
relevantes = d[d['Relevância'] == 'R']
irrelevantes = d[d['Relevância'] == 'I']

#Separa o dataframe frase por frase e adiciona na lista relevantes ou irrelevantes
frases_relevantes =[]
frases_irrelevantes = []

for linha in relevantes['Treinamento']:
    linha = linha.split()
    frases_relevantes.append(linha)
    
for linha in irrelevantes['Treinamento']:
    linha = linha.split()
    frases_irrelevantes.append(linha)
    
# Separa o número de palavras em cada frase
n_palavra_relevante =[]
n_palavra_irrelevante=[]

for frase in frases_relevantes:
    x = len(frase)
    n_palavra_relevante.append(x)
    
for frase in frases_irrelevantes:
    y = len(frase)
    n_palavra_irrelevante.append(y)
    
# Total de palavras no dataframe, em cada categoria    
total_relevantes = sum(n_palavra_relevante)
total_irrelevantes = sum(n_palavra_irrelevante)

palavras_relevantes = sum(frases_relevantes, [])
palavras_irrelevantes = sum(frases_irrelevantes, [])


# Conta as possibilidades de palavra (cada palavra que aparece, sem repetição)
total_de_palavras = []

for palavra in palavras_relevantes:
    if palavra not in total_de_palavras:
        total_de_palavras.append(palavra)

for palavra in palavras_irrelevantes:
    if palavra not in total_de_palavras:
        total_de_palavras.append(palavra)        
        
total_de_palavras = len(total_de_palavras)
 
    
# Cria um dicionário de recorrência de cada palavra, em cada categoria
palavras_rel_recorrencia = dict(Counter(palavras_relevantes))
palavras_irr_recorrencia = dict(Counter(palavras_irrelevantes))


# Cria dois dicionários cujo o valor de cada palavra é a sua probabilidade de ocorrência em frases relevantes e irrelevantes
dic_prob_rel ={}
dic_prob_irrel ={}

for palavra in palavras_rel_recorrencia:
    p = ((palavras_rel_recorrencia[palavra])+1)/(total_relevantes+total_de_palavras)
    dic_prob_rel[palavra] = p
    
for palavra in palavras_irr_recorrencia:
    p = ((palavras_irr_recorrencia[palavra])+1)/(total_irrelevantes+total_de_palavras)
    dic_prob_irrel[palavra] = p
    


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [207]:
# Leitura da base de dados para teste e remoção de caracteres de pontuação.
d_teste = pd.read_excel('Ovomaltine_teste.xlsx')
d_teste['Teste'] = d_teste['Teste'].apply(remove_punctuation)
d_teste = pd.concat([d_teste['Teste'],d_teste['Unnamed: 1']], axis=1)
d_teste.columns = ['Teste','Relevância']

# Separa as frases do dataframe de teste
frases_teste = []

for linha in d_teste['Teste']:
    linha = linha.split()
    frases_teste.append(linha)

# Calcula o produto das probabilidades das palavras de cada frase.
# Se o produto das probabilidades para relevante for maior que para irrelevantes, adiciona R em relevancia, caso contrário, I
relevancia = []
for i in range(0,len(frases_teste)):
    for palavra in frases_teste[i]:
        if palavra in dic_prob_rel:
            produto=1
            produto = produto* dic_prob_rel[palavra]
        if palavra in dic_prob_irrel:
            produto2 = 1
            produto2 = produto2* dic_prob_irrel[palavra]    
    if produto>produto2:
        relevancia.append('R')
    if produto2>produto:
        relevancia.append('I')
    
# Cria um dataframe para os dados do classificador    
dados_classificador = {'Relevância(classificador)':pd.Series(relevancia)}
dados_classificador = pd.DataFrame(dados_classificador)

# Junta os dados para comparação
df = pd.concat([d_teste['Teste'],d_teste['Relevância'],dados_classificador], axis = 1)

df

,Teste,Relevância,Relevância(classificador)
0,vou fazer um brigadeiro com ovomaltine ♥♥,R,R
1,pelo menos tenho meu ovomaltine e pão com mort...,I,R
2,milk shake trufado de ovomaltine eu te amo,R,I
3,minha meta neste casamento é ficar muito loco ...,I,R
4,gostei de um vídeo @youtube https://t.co/ar4tg...,I,I
5,q vontade de tmr um milkshake de ovomaltine,R,R
6,um sorvete de ovomaltine cairia bem agora com ...,R,R
7,vou tomar meu ovomaltine e dormir bem pq aind...,I,R
8,gostei de um vídeo @youtube https://t.co/h6ual...,I,I
9,ovomaltine com leite bem gelado é bom dmssssss...,R,R


In [208]:
# Cria uma crosstab para comparação
ct = ct1=pd.crosstab(df['Relevância'], df['Relevância(classificador)'], margins=True, rownames=['Relevância'], colnames=['Relevância (classificador)'])
ct

Relevância (classificador),I,R,All
Relevância,,,
I,68,55,123
R,20,57,77
All,88,112,200


In [209]:
# Porcentagem de RELEVANTES VERDADEIROS

p_acerto_relevante = (56/76)*100
print(p_acerto_relevante,'%')

73.68421052631578 %


In [210]:
# Porcentagem de RELEVANTES FALSOS

p_erro_relevante = 100-p_acerto_relevante
print(p_erro_relevante,'%')

26.31578947368422 %


In [211]:
# Porcentagem de IRRELEVANTES VERDADEIROS

p_acerto_irrelevante = (69/124)*100
print(p_acerto_irrelevante,'%')

55.64516129032258 %


In [212]:
# Porcentagem de IRRELEVANTES FALSOS

p_erro_irrelevante = 100-p_acerto_irrelevante
print(p_erro_irrelevante,'%')

44.35483870967742 %


## Adicionando categorias intermediárias:

In [213]:
# Adiciona categorias intermediárias (Muito Relevante, Relevante, Neutro, Irrelevante e Muito Irrelevante)
produtos = []
relevancia2 = []

for i in range(0,len(frases_teste)):
    for palavra in frases_teste[i]:
        if palavra in dic_prob_rel:
            produto=1
            produto = produto* dic_prob_rel[palavra]
        if palavra in dic_prob_irrel:
            produto2 = 1
            produto2 = produto2* dic_prob_irrel[palavra]
    p = produto-produto2
    produtos.append(p)
    if p>0.02:
        relevancia2.append('Muito Relevante')
    if p<0.025 and p>0.005:
        relevancia2.append('Relevante')
    if p<0.005 and p>-0.0001:
        relevancia2.append('Neutro')
    if p<-0.0001 and p>-0.025:
        relevancia2.append('Irrelevante')
    if p<-0.025:
        relevancia2.append('Muito Irrelevante')

In [214]:
# Cria dataframe com os novos dados, e junta com a base de testes
dados_classificador2 = {'Relevância(classificador)':pd.Series(relevancia2)}
dados_classificador2 = pd.DataFrame(dados_classificador2)

df2 = pd.concat([d_teste['Teste'],d_teste['Relevância'],dados_classificador2], axis = 1)

df2 = df2.dropna()
df2

,Teste,Relevância,Relevância(classificador)
0,vou fazer um brigadeiro com ovomaltine ♥♥,R,Relevante
1,pelo menos tenho meu ovomaltine e pão com mort...,I,Neutro
2,milk shake trufado de ovomaltine eu te amo,R,Irrelevante
3,minha meta neste casamento é ficar muito loco ...,I,Relevante
4,gostei de um vídeo @youtube https://t.co/ar4tg...,I,Irrelevante
5,q vontade de tmr um milkshake de ovomaltine,R,Relevante
6,um sorvete de ovomaltine cairia bem agora com ...,R,Neutro
7,vou tomar meu ovomaltine e dormir bem pq aind...,I,Muito Relevante
8,gostei de um vídeo @youtube https://t.co/h6ual...,I,Irrelevante
9,ovomaltine com leite bem gelado é bom dmssssss...,R,Relevante


In [215]:
# Cria crosstab para comparação
ct2 = pd.crosstab(df2['Relevância'], df2['Relevância(classificador)'], margins=True, rownames=['Relevância'], colnames=['Relevância (classificador)'])
ct2

Relevância (classificador),Irrelevante,Muito Irrelevante,Muito Relevante,Neutro,Relevante,All
Relevância,,,,,,
I,65,1,5,28,24,123
R,18,2,1,29,27,77
All,83,3,6,57,51,200


## Conclusão

Como visto na tabela acima, o classificador considera muitos dos comentários marcados como 'Irrelevantes' são marcados como 'Relevantes'. Isso se deve a um particular rigor ao classificar manualmente os tweets que seriam utilizados como teste. Entretanto, o classificador tem uma porcentagem de acerto alta em 'Relevantes verdadeiros', e acerta mais da metade nos marcados como 'Irrelevantes verdadeiros'. <br/>

O classificador Naive-Bayes, no entanto, possui algumas limitações. Frases de dupla negação e sarcasmo podem confundi-lo, justamente pelo mesmo considerar cada palavra independente do contexto da frase. Portanto, se uma frase irônica, por exemplo, possuir **sentido relevante**, mas utilizar palavras que aparecem mais nas frases irrelevantes, será marcada como irrelevante. Daí vem o nome 'Naive', que do inglês, quer dizer 'Ingênuo'. <br />

Outra limitação do classificador diz respeito à base de dados para treinamento. Não é possível alimentá-la utilizando o próprio
classificador. Isto se deve ao fato de as probabilidades de cada palavra serem relevantes ou irrelevantes variam de acordo com o tema/produto, de modo que o classificador acima só se aplica ao produto Ovomaltine. Para que outros assuntos sejam classificados é necessário criar uma nova base de dados para Treinamento, classificando frase por frase. <br />

O uso do classificador bayesiano, no entanto, pode ser muito útil, não só para classificar produtos, mas em outras esferas como política. Através dele, é possível saber o que usuários do Twitter falam sobre um determinado candidato ou determinado problema na/no cidade/estado/país, e assim direcionar o marketing. Esferas governamentais também podem utilizar o classificador para entender os problemas mais relevantes para a população, e assim, criar estratégias para resolvê-los. <br />

Para a empresa, é essencial continuar com o projeto justamente por ser importante saber a opinião dos consumidores sobre o produto. Além disso, no caso do Ovomaltine, muitos dos usuários o consumem com outros produtos, como na forma de milkshake, com brigadeiro, etc. Isso cria novas possibilidades para produtos derivados, e consequentemente, a expansão do mercado consumidor. O classificador foi aplicado para apenas 500 tweets. Com uma expansão, passaria a analisar mais tweets, colher mais dados, e criar um classificador mais preciso, gerando ainda mais informações e feedbacks dos consumidores para a empresa. <br />

